In [34]:
import yaml
import sys
import os
from pydub import AudioSegment
from PIL import Image
import zipfile
from shutil import make_archive as shutil

In [35]:
zipPath = './3602 Bethoven v3.zip'
zipFile = os.path.basename(zipPath)
zipFileBase = os.path.splitext(zipFile)[0]
os.mkdir(zipFileBase)
storyZip = zipfile.ZipFile('./3602 Bethoven v3.zip', 'r')
storyZip.extractall(zipFileBase)

FileExistsError: [Errno 17] File exists: '3602 Bethoven v3'

In [32]:
f = open(zipFileBase+'/'+'story.yml')
storyDict = yaml.safe_load(f)
f.close()

In [33]:
# iterate thru rooms
for room in storyDict['rooms']:
    print('---')
    print(room['name'])
    roomFolder = room['uuid'] + '/'
    
    # recompress background image
    backgroundPath = zipFileBase + '/' + roomFolder + room['image']
    background = Image.open(backgroundPath)
    background.save(backgroundPath,quality=80,optimize=True)
    print("Optimizing: ", room['image'])
    
    # iterate thru audio hotspots
    for clip in room['clips']:
        clipPath = zipFileBase + '/' + roomFolder + clip['file']
        clipPathMp3 = os.path.splitext(clipPath)[0]+'.mp3'
        print("MP3-ing: ", clip['file'])
        AudioSegment.from_wav(clipPath).export(clipPathMp3, format="mp3")
        os.remove(clipPath)
        
    # check for narration
    narrateIntro = room['narrator']['intro']
    if (narrateIntro):
        narratePath = zipFileBase + '/' + roomFolder + room['narrator']['intro']
        narratePathMp3 = os.path.splitext(narratePath)[0]+'.mp3'
        #print(narratePath+ " " + narratePathMp3)
        print("MP3-ing: ", room['narrator']['intro'])
        AudioSegment.from_wav(narratePath).export(narratePathMp3, format="mp3")

---
Deck
Optimizing:  R0010941.JPG
MP3-ing:  1381242529.wav


FileNotFoundError: [Errno 2] No such file or directory: '3602 Bethoven v3/237691876/1381242529.wav'

In [30]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [31]:
zipf = zipfile.ZipFile(zipFileBase+"_slim.zip",'w',zipfile.ZIP_DEFLATED)
zipdir(zipFileBase,zipf)
zipf.close()